In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')
sys.path.append('../../')

from dataset import CocoDetection, train_transforms, val_transforms, test_transforms
from visualize import visualize
# from rcnn_model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from models.swin import *

In [2]:
from models.detection.backbone_utils import swin_fpn_backbone, _validate_trainable_layers
from ops.feature_pyramid_network import LastLevelP6P7, LastLevelMaxPool
from models.detection.retinanet import RetinaNet
from torch.hub import load_state_dict_from_url
from models.detection.anchor_utils import AnchorGenerator
# from models.detection.backbone_utils import mobilenet_backbone

In [3]:
def retinanet_swin_t_fpn(pretrained=False, progress=True,
                           num_classes=91, pretrained_backbone=False, trainable_backbone_layers=None, **kwargs):
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 5, 3)

    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False
        
    anchor_sizes = ((32, 64, 128, 256, 512), ) * 5
    aspect_ratios = ((0.5, 0.75, 1.0, 1.5, 2.0),) * len(anchor_sizes)
    rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios)
    
    # skip P2 because it generates too many anchors (according to their paper)
    backbone = swin_fpn_backbone('swin_t', pretrained_backbone, returned_layers=[2, 3, 4],
                                   extra_blocks=LastLevelP6P7(256,256), trainable_layers=trainable_backbone_layers)
    
    model = RetinaNet(backbone, num_classes, anchor_generator=rpn_anchor_generator, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['retinanet_resnet50_fpn_coco'],
                                              progress=progress)
        model.load_state_dict(state_dict)
        overwrite_eps(model, 0.0)
    return model

In [4]:
NUM_CLASS = 91
IMG_SIZE = 448*2
model = retinanet_swin_t_fpn(pretrained=False, min_size=IMG_SIZE, max_size=IMG_SIZE, num_classes=NUM_CLASS)

device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

/home/beomgon/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272128894/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


return_layers {'layer2': '0', 'layer3': '1', 'layer4': '2'}
model is loaded to gpu


In [5]:
# model

In [6]:
import easydict 
args = easydict.EasyDict({ "batch_size": 4, 
                          "epochs": 90, 
                          "data": 0, 
                          'lr':0.002,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':16,
                         'print_freq':1000,
                         'output_dir':'../trained_models/retinanet_swin_t_fpn/'})

In [7]:
from pathlib import Path
path = Path(args.output_dir.split('checkpoint')[0])
path.mkdir(parents=True, exist_ok=True)  

In [8]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
GPU_NUM = args.gpu # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

3
cuda:0


In [9]:
from dataset import CocoDetection, train_transforms, val_transforms, test_transforms
train_dataset = CocoDetection(root='/home/beomgon/Dataset/scl/', annFile='../../data/train.json', 
                              transforms=train_transforms)
test_dataset = CocoDetection(root='/home/beomgon/Dataset/scl/', annFile='../../data/test.json', 
                              transforms=val_transforms)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [10]:
image, target = next(iter(train_dataset))
target
                    

{'boxes': tensor([[667.1875, 353.0625, 743.3125, 423.5000]]),
 'category_id': tensor([1]),
 'labels': tensor([1]),
 'image_id': tensor([1]),
 'area': tensor([5362.0547]),
 'iscrowd': tensor([0])}

In [11]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [12]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=args.lr, weight_decay=args.weight_decay)
optimizer = torch.optim.SGD(
       params, lr=args.lr, momentum=args.momentum, weight_decay=args.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30, 45, 60, 75], 
                                                    gamma=0.5)

In [ ]:
from engine import train_one_epoch

start_time = time.time()
for epoch in range(args.epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, args.print_freq)
    lr_scheduler.step()
    
    if epoch > 60 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

    if epoch > 5 and epoch % 5 == 0 :
        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [   0/1549]  eta: 1:28:01  lr: 0.000004  loss: 2.3234 (2.3234)  classification: 1.6080 (1.6080)  bbox_regression: 0.7155 (0.7155)  time: 3.4096  data: 1.9179  max mem: 14001
Epoch: [0]  [1000/1549]  eta: 0:09:39  lr: 0.002000  loss: 1.8649 (1.9831)  classification: 1.2687 (1.3652)  bbox_regression: 0.5976 (0.6179)  time: 1.0508  data: 0.0166  max mem: 14337
Epoch: [0]  [1548/1549]  eta: 0:00:01  lr: 0.002000  loss: 1.5451 (1.9424)  classification: 0.9872 (1.3316)  bbox_regression: 0.5677 (0.6107)  time: 1.0212  data: 0.0160  max mem: 14345
Epoch: [0] Total time: 0:27:10 (1.0524 s / it)
Epoch: [1]  [   0/1549]  eta: 1:21:58  lr: 0.002000  loss: 1.7217 (1.7217)  classification: 1.0586 (1.0586)  bbox_regression: 0.6631 (0.6631)  time: 3.1751  data: 2.0504  max mem: 14345
Epoch: [1]  [1000/1549]  eta: 0:09:38  lr: 0.002000  loss: 1.4850 (1.6467)  classification: 0.8593 (1.0538)  bbox_regression: 0.5874 (0.5929)  time: 1.0491  data: 0.0159  max mem: 14345
Epoch: [1]  [1548/1549]

In [ ]:
model